In [1]:
import time
import pandas as pd
from zomatopy import Zomato
#get_categories
#get_city_ID
#get_city_name
#get_collections
#get_cuisines
#get_establishment_types
#get_nearby_restaurants
#get_restaurant
#restaurant_search

zconfig = {"user_key":"2dde89df168a9311ecd9f1459126144c"}

### READ DATA

In [2]:
zcities_df = pd.read_csv('../data/kin_nov2/zcities_cleaned.csv') # cleaned zomato cities
cstates_df = pd.read_csv('../data/kin_nov2/top_population_state.csv') # census top 10 migrant populations
cuisines_df = pd.read_csv('../data/kin_nov2/cuisine_ids.csv') # merged cuisines and populations

In [3]:
zcities_df.drop('Unnamed: 0', axis=1, inplace=True)
zcities_df.head(5)

,zcity_id,city_query,state_code
0,276,Dallas TX,TX
1,277,Houston TX,TX
2,278,Austin TX,TX
3,279,Seattle WA,WA
4,280,New York NY,NY


In [4]:
cstates_df.head(5)

,state,state_code,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,ALABAMA,AL,Mexico,Guatemala,Korea,India,China,Germany,Vietnam,Philippines,Honduras,Canada
1,ALASKA,AK,Philippines,Mexico,Korea,Thailand,Canada,Russia,Germany,Dominican Republic,"Oceania, nec",Laos
2,ARIZONA,AZ,Mexico,Canada,India,Philippines,China,Vietnam,Germany,Guatemala,Korea,Iraq
3,ARKANSAS,AR,Mexico,El Salvador,India,"Oceania, nec",Guatemala,Vietnam,China,Philippines,Laos,Germany
4,CALIFORNIA,CA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan


In [5]:
cuisines_df.head(5)

,population,cuisine_names,cuisine_id
0,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,Canada,Canada,381
2,India,India,148
3,China,"China, Sichuan","25,128"
4,Vietnam,Vietnam,99


### QUERY DATAFRAME

In [6]:
query_df = zcities_df.copy()
query_df = query_df.merge(cstates_df, on='state_code', how='left')
perc = query_df.population1.isna().sum()/len(query_df)
print(f'missing state merge percentage {perc}')

missing state merge percentage 0.0


In [7]:
query_df.head(5)

,zcity_id,city_query,state_code,state,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,276,Dallas TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
1,277,Houston TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
2,278,Austin TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
3,279,Seattle WA,WA,WASHINGTON,Mexico,India,Philippines,China,Vietnam,Korea,Canada,Ukraine,Russia,Germany
4,280,New York NY,NY,NEW YORK,Dominican Republic,China,Jamaica,Mexico,Ecuador,Guyana,India,Haiti,El Salvador,Colombia


In [8]:
query_df = pd.wide_to_long(query_df, stubnames='population',
                           i=['zcity_id', 'state_code'], j='top')
query_df = query_df.reset_index()
query_df.head(10)

,zcity_id,state_code,top,state,city_query,population
0,276,TX,1,TEXAS,Dallas TX,Mexico
1,276,TX,2,TEXAS,Dallas TX,India
2,276,TX,3,TEXAS,Dallas TX,El Salvador
3,276,TX,4,TEXAS,Dallas TX,Vietnam
4,276,TX,5,TEXAS,Dallas TX,Honduras
5,276,TX,6,TEXAS,Dallas TX,China
6,276,TX,7,TEXAS,Dallas TX,Philippines
7,276,TX,8,TEXAS,Dallas TX,Guatemala
8,276,TX,9,TEXAS,Dallas TX,Nigeria
9,276,TX,10,TEXAS,Dallas TX,Pakistan


In [9]:
query_df = query_df.reset_index()

In [10]:
query_df = query_df.merge(cuisines_df, on='population', how='left')
perc = query_df.cuisine_id.isna().sum()/len(query_df)
print(f'missing cuisine_id percentage {perc}')
check_missings = query_df[query_df.cuisine_id.isna()].population
check_missings = check_missings.unique()
query_df.head(10)

missing cuisine_id percentage 0.0


,index,zcity_id,state_code,top,state,city_query,population,cuisine_names,cuisine_id
0,0,276,TX,1,TEXAS,Dallas TX,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,1,276,TX,2,TEXAS,Dallas TX,India,India,148
2,2,276,TX,3,TEXAS,Dallas TX,El Salvador,El Salvador,601
3,3,276,TX,4,TEXAS,Dallas TX,Vietnam,Vietnam,99
4,4,276,TX,5,TEXAS,Dallas TX,Honduras,South America,972
5,5,276,TX,6,TEXAS,Dallas TX,China,"China, Sichuan","25,128"
6,5,276,TX,6,TEXAS,Dallas TX,China,"Hong Kong, Dim Sum","121, 411"
7,6,276,TX,7,TEXAS,Dallas TX,Philippines,Philippines,112
8,7,276,TX,8,TEXAS,Dallas TX,Guatemala,Guatemala,"972,136"
9,8,276,TX,9,TEXAS,Dallas TX,Nigeria,Nigeria,152


In [11]:
check_missings

array([], dtype=object)